### CS 585 Assignment 2
> **Name**:   Jiayin Fan<br>
> **Email**:      jfan22@bu.edu<br>
> **Student ID**: U80694562<br>

In [57]:
import numpy as np
import cv2

In [58]:
def bgDifferencing(region, background):
    diff = cv2.absdiff(np.uint8(background), region)
    _, thresholded = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        segmented = max(contours, key=cv2.contourArea)
        return (thresholded, segmented)

In [59]:
def recognize_gesture(segmented):
    # Ensure that segmented is the largest contour and is valid for processing
    # Calculate the convex hull with returnPoints=True
    hull = cv2.convexHull(segmented, returnPoints=True)
    
    # Now, convert hull points back to format expected by convexityDefects
    hull_indices = cv2.convexHull(segmented, returnPoints=False)
    if len(hull_indices) > 3:
        try:
            defects = cv2.convexityDefects(segmented, hull_indices)
            if defects is not None:
                cnt = 0  # Counter for defects
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(segmented[s][0])
                    end = tuple(segmented[e][0])
                    far = tuple(segmented[f][0])
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c))  # cosine theorem
                    if angle <= np.pi / 2:  # angle less than 90 degree, treat as a defect
                        cnt += 1
                if cnt > 0:
                    return "Hand Open"
                else:
                    return "Fist"
        except Exception as e:
            print(f"Error in calculating defects: {e}")
    return "Unknown"


In [60]:
# Main loop to get input from the camera
def captureCamera():

    # Init frame counter, bg indicator and caliration time
    frames_elapsed = 0
    background = None
    CALIBRATION_TIME = 30
    BG_WEIGHT = 0.5 

    # Define region of interest bounds
    roi_x, roi_y, roi_width, roi_height = 0, 0, 600, 600

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        # Filp the video for easier testing; turn it into grayscale and blur it
        frame = cv2.flip(frame, 1)

        # Increment the frames_elapsed counter
        frames_elapsed += 1

        # Extract the ROI from the frame
        roi = frame[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_roi = cv2.GaussianBlur(gray_roi, (7, 7), 0)
        
        # Process the ROI with gesture recognition methods
        if frames_elapsed < CALIBRATION_TIME:
            if background is None:
                background = np.float32(gray_roi)
            else:
                cv2.accumulateWeighted(gray_roi, background, BG_WEIGHT)
            frames_elapsed += 1
            cv2.putText(frame, 'Calibrating...', (roi_x + 10, roi_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            # Background subtraction
            hand = bgDifferencing(gray_roi, background)

            # Gesture recognition
            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(roi, [segmented + (roi_x, roi_y)], -1, (0, 255, 0))
                gesture = recognize_gesture(segmented)
                cv2.putText(frame, gesture, (roi_x, roi_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.imshow("Thresholded", thresholded)
        
        # Display the ROI
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x+roi_width, roi_y+roi_height), (0, 255, 0), 2)
        
        cv2.imshow('Gesture Recognition', frame)
        
        # Stop capturing when pressing "q"
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [61]:
captureCamera()

Error in calculating defects: OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'convexityDefects'

Error in calculating defects: OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'convexityDefects'

Error in calculating defects: OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'convexityDefects'

Error in calculating defects: OpenCV(4.9.0) /Users/runner/wor